In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 2.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30315 sha256=64c04f7d1ed53c568bf5f37944f44d4b001cf84f017accf1b68fa9f33b3fdc0e
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=3d32d4759989c5078c3316aa11233081b872a182cf60bb6c7afa4609f4f9cc97
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=71eace5a8a1528fe5d3674e16aa8c54a697a331460f19c0e6da6d54d99a24c4e
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 4.7MB/s 


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as up
import pandas as pd
from tensorflow.keras import layers
import bert
import random

from google.colab import drive


In [3]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

# string processing
import re
from keras.utils.np_utils import to_categorical
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics


Using TensorFlow backend.


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [8]:

df_merge_quality = pd.read_csv(
    "/content/drive/My Drive/NLP_Udemy/data/US_patent_abstract_50000_2015_with_title_1_5y.csv",
 #   header=None,
 #   names=cols,
    engine="python",
    encoding="latin1"
)

In [9]:
df = df_merge_quality[['claims_text', 'quality_rank']]
df

,claims_text,quality_rank
0,What is claimed is: \n \n 1. A user ...,1
1,I claim: \n \n 1. A mechanism for as...,0
2,We claim: \n \n 1. An aircraft landi...,1
3,1. A method for culturing primate pluripotent ...,1
4,What is claimed is: \n \n 1. A metho...,0
...,...,...
49995,What is claimed is: \n \n 1. A sched...,0
49996,What is claimed is: \n \n 1. An appa...,0
49997,What is claimed is: \n \n 1. An imag...,1
49998,I claim: \n \n 1. A method comprisin...,0


### Prep for Data

In [10]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [11]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [12]:
claims = []
sentences = list(df['claims_text'])
for sen in sentences:
    claims.append(preprocess_text(str(sen)))

In [13]:
claims[0]

'What is claimed is user equipment UE device the UE device comprising an antenna for performing wireless communications with base station memory which stores plurality of sets of communication scenario information for each of plurality of UE communication scenarios wherein each of the stored plurality of sets of communication scenario information is based on two or more of receiver type multiple input multiple output MIMO scheme and amount of Doppler shift wherein each set of communication scenario information is useable in generating channel quality indicator for respective communication scenario processor configured to determine current communication scenario of the UE device during operation of the UE wherein determining the current communication scenario is based on the two or more of receiver type MIMO scheme and amount of Doppler shift select first set of communication scenario information from the plurality of sets of communication scenario information based on the determined cu

In [14]:
#data_labels = df.quality_rank.values
#data_labels

array([1, 0, 1, ..., 1, 0, 0])

In [15]:
# Prepare for data label
data_labels = to_categorical(df.quality_rank.values)
data_labels

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [16]:
# BERT tokenizer
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [17]:
def encode_sentence(sent, max_seq_length):
    if len(sent) <= max_seq_length:
        return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]
    else: # BERT limited to 512 tokens
        return ["[CLS]"] + tokenizer.tokenize(sent)[-max_seq_length:] + ["[SEP]"] # change to last n words

In [18]:
# BERT takes maximum 512 sequence

MAX_SEQ_LEN=510

tokenized_claims = [encode_sentence(sentence, MAX_SEQ_LEN) for sentence in claims]

KeyboardInterrupt: ignored

In [ ]:
tokenized_claims[:2]

In [ ]:
np.array(tokenized_claims).shape

In [ ]:
# Prepre the 3 inputs required by BERT deriving from the original data

def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)  # if not equal [PAD] then assign 1. 

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # turns 1 into 0 and vice versa
    return seg_ids

Below to create padded batches (so there could have different sentence length between batches, but will be same sentence length within batch) this is to save processing memory.

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(tokenized_claims)]

In [ ]:
# split to train and test

data_with_len_train = data_with_len[:35000]
data_with_len_dev = data_with_len[35000:45000]

In [ ]:
data_with_len_train[0]

In [ ]:
np.array(data_with_len_train).shape

In [ ]:
np.unique(np.array(data_with_len_train)[:, 2], return_counts = True)
# majority are in 512 and above

In [32]:
def to_batch(data_with_len, BATCH_SIZE):
    data_with_len = sorted(data_with_len, key=lambda x: x[2])
    sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len] 
    
    train_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))
    

    train_batched = train_dataset.padded_batch(BATCH_SIZE, # this is the pre-processed input to feed into BERT embedding layer and DCNN model
                                       padded_shapes=((3, 512), [2]), # oh, here is the key, need to change from () to [2] now that I pass in the one-hot label!
                                       padding_values=(0, 0)) # drop_remainder = True (optional)
    return train_batched

In [33]:
# prepare the batches for train set:

BATCH_SIZE = 32

train_batched = to_batch(data_with_len_train, BATCH_SIZE)

In [34]:
train_batched # note: if above use drop_remainder = True, then => shapes: ((32, 3, None), (32, 2)), types: (tf.int32, tf.int32)>  because it would only be "None" if any single item is inconsistent in shape with the rest

<PaddedBatchDataset shapes: ((None, 3, 512), (None, 2)), types: (tf.int32, tf.int32)>

In [35]:
next(iter(train_batched))


(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
 array([[[  101,  7328,  2383, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  2054,  2003, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  2057,  4366, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        ...,
 
        [[  101,  1996, 11028, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  2054,  2003, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  2054,  2003, ...,     0,     

In [36]:
# prepare the batches for evaluation set:
test_batched = to_batch(data_with_len_dev, BATCH_SIZE)

In [37]:
next(iter(test_batched))

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
 array([[[  101,  1996, 11028, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  2054,  2003, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  1996, 11028, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        ...,
 
        [[  101,  1996, 11028, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  2057,  4366, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  4118,  1997, ...,     0,     

In [38]:
# double check the label distribution
# Validation set
count = 0
sum_positive = 0
for element in test_batched:
    count += len(element[1])
    sum_positive += sum(element[1])

print(f'Sample Count:{count}, Positive Label Count:{sum_positive}, Positive Class Ratio:{sum_positive/count}, Negative Class Ratio = {1-sum_positive/count}')


Sample Count:10000, Positive Label Count:[5898 4102], Positive Class Ratio:[0.5898 0.4102], Negative Class Ratio = [0.4102 0.5898]


In [39]:
# double check the label distribution
# Train set
count = 0
sum_positive = 0
for element in train_batched:
    count += len(element[1])
    sum_positive += sum(element[1])

print(f'Sample Count:{count}, Positive Label Count:{sum_positive}, Positive Class Ratio:{sum_positive/count}, Negative Class Ratio = {1-sum_positive/count}')


Sample Count:35000, Positive Label Count:[20815 14185], Positive Class Ratio:[0.59471429 0.40528571], Negative Class Ratio = [0.40528571 0.59471429]


### Model Building

In [40]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=5,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=50,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=100,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=2,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)

        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

### Training

In [41]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

# BATCH_SIZE = 32  # initialize above
NB_EPOCHS = 5

In [42]:
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [43]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [44]:
checkpoint_path = "/content/drive/My Drive/NLP_Udemy/data/ckpt_BERT_CNN_US_5years_last_510_words_50k/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [45]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [45]:
# fitting time
Dcnn.fit(train_batched,
         validation_data = (test_batched),
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

Epoch 1/5
   1094/Unknown - 1760s 2s/step - loss: 0.6734 - accuracy: 0.6169Checkpoint saved at /content/drive/My Drive/NLP_Udemy/data/ckpt_BERT_CNN_US_5years_last_510_words_50k/.
1094/1094 [==============================] - 2237s 2s/step - loss: 0.6734 - accuracy: 0.6169 - val_loss: 0.6609 - val_accuracy: 0.6064
Epoch 2/5
1094/1094 [==============================] - 2236s 2s/step - loss: 0.6296 - accuracy: 0.6454 - val_loss: 0.6518 - val_accuracy: 0.6148
Epoch 3/5
1094/1094 [==============================] - 2235s 2s/step - loss: 0.6118 - accuracy: 0.6600 - val_loss: 0.6597 - val_accuracy: 0.6030
Epoch 4/5
1094/1094 [==============================] - 2235s 2s/step - loss: 0.5919 - accuracy: 0.6788 - val_loss: 0.6733 - val_accuracy: 0.6034
Epoch 5/5
1094/1094 [==============================] - 2240s 2s/step - loss: 0.5679 - accuracy: 0.7022 - val_loss: 0.6928 - val_accuracy: 0.6057


In [46]:
# modify from original to_batch in order to not sort to different order because for ensemble we need to preserve the same order among all models
def prediction_to_batch(data_with_len, BATCH_SIZE):
    # data_with_len = sorted(data_with_len, key=lambda x: x[2])
    sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len] 
    
    train_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))
    

    train_batched = train_dataset.padded_batch(BATCH_SIZE, # this is the pre-processed input to feed into BERT embedding layer and DCNN model
                                       padded_shapes=((3, 512), [2]), # make from None to 512 fixed dimension
                                       padding_values=(0, 0)) # drop_remainder = True (optional)
    return train_batched

In [47]:
# prepare the batches for evaluation set:
test_batched_prediction = prediction_to_batch(data_with_len_dev, 32)

In [48]:
next(iter(test_batched_prediction))

(<tf.Tensor: shape=(32, 3, 512), dtype=int32, numpy=
 array([[[  101,  1997,  1996, ...,  5492,  6943,   102],
         [    1,     1,     1, ...,     1,     1,     1],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  2054,  2003, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  2000,  2055, ...,  8681, 16216,   102],
         [    1,     1,     1, ...,     1,     1,     1],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        ...,
 
        [[  101,  6058,  3538, ...,  7277,  3430,   102],
         [    1,     1,     1, ...,     1,     1,     1],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  4860, 13907, ..., 17785, 15273,   102],
         [    1,     1,     1, ...,     1,     1,     1],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101, 15176, 10984, ...,  2140,  498

In [49]:
pred_test = Dcnn.predict(test_batched_prediction)
pred_test

array([[0.5658154 , 0.4443321 ],
       [0.581872  , 0.4347374 ],
       [0.36238575, 0.6277524 ],
       ...,
       [0.6997249 , 0.29793364],
       [0.96749085, 0.02547945],
       [0.38488117, 0.6237045 ]], dtype=float32)

In [50]:
pred_test.shape

(10000, 2)

In [51]:
pred_test.shape

(10000, 2)

In [52]:
np.savetxt('/content/drive/My Drive/NLP_Udemy/data/BERT_CNN_5yr_claims_last_510_words_50k_dev_prob.csv', pred_test)

In [53]:
predicted = [np.argmax(pred) for pred in 
             pred_test]

In [22]:
y_test_binary = df['quality_rank'][35000:45000].values


In [23]:
## Accuracy, Precision, Recall
accuracy = metrics.accuracy_score(y_test_binary, predicted)
auc = metrics.roc_auc_score(y_test_binary, predicted)  # predicted_prob), check doc, seems the second argument required to be shape (n_samples,) for binary case 
                            #multi_class="ovr") # check documentation and seems "ovr" not good for only binary target class
print("Accuracy:",  round(accuracy,3))
print("Auc:", round(auc,3))
print("Detail:")
print(metrics.classification_report(y_test_binary, predicted))



Accuracy: 0.606
Auc: 0.602
Detail:
              precision    recall  f1-score   support

           0       0.68      0.62      0.65      5898
           1       0.52      0.58      0.55      4102

    accuracy                           0.61     10000
   macro avg       0.60      0.60      0.60     10000
weighted avg       0.61      0.61      0.61     10000

